# LLM reasoning pop quiz

Do open-sourced LLMs have the reasoning prowess of their closed-sourced siblings?


## Table of Contents

1. Setup
2. Read yaml config
3. Load model
4. Run the quiz

## Setup

In [1]:
# detailed information on the GPU

!nvidia-smi

Fri Nov 17 04:40:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    42W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/daniel-furman/LLM-reasoning-pop-quiz.git

Cloning into 'LLM-reasoning-pop-quiz'...
remote: Enumerating objects: 665, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 665 (delta 22), reused 35 (delta 11), pack-reused 613
Receiving objects: 100% (665/665), 359.98 KiB | 18.95 MiB/s, done.
Resolving deltas: 100% (369/369), done.


In [3]:
# install necessary libraries
import os

os.chdir("/content/LLM-reasoning-pop-quiz")
!pip install -q -U -r requirements.txt
os.chdir("..")

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.2/220.2 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 107.2 MB/

In [4]:
# import libraries

import transformers
import torch
import time
import yaml

# import helpers

from boilers import hf_llm_boiler

In [5]:
# set the seed

transformers.set_seed(42)

In [6]:
# print GPU available memory

free_in_GB = int(torch.cuda.mem_get_info()[0] / 1024**3)
max_memory = f"{free_in_GB-2}GB"

n_gpus = torch.cuda.device_count()
max_memory = {i: max_memory for i in range(n_gpus)}
max_memory

{0: '36GB'}

## Read in the yaml config for the run

In [7]:
with open("/content/LLM-reasoning-pop-quiz/configs/pop_quiz.yml", "r") as file:
    pop_quiz = yaml.safe_load(file)
pop_quiz

{'prompts': {'zero_shot': ["A juggler has 16 balls. Half of the balls are golf balls and half of the golf balls are blue. How many blue golf balls are there?\nLet's respond succinctly and work this out in a step by step way to be sure we have the right answer.",
   "Daniel is in need of a haircut. His barber works Mondays, Wednesdays, and Fridays. So, Daniel went in for a haircut on Sunday. Does this make logical sense?\nLet's respond succinctly and work this out in a step by step way to be sure we have the right answer.",
   "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?\nLet's respond succinctly and work this out in a step by step way to be sure we have the right answer.",
   "Roger has 3 children. Each of his kids invited 4 of their friends to come to the birthday party. All of the friends came to the party. How many children are at the party?\nLet's respond succinctly and work this out in a step by step way to be sure we

## Load the model

In [8]:
# load model
# this cell will take a long time, to avoid: deploy the LLM as an API inference endpoint

model_id = "dfurman/Mistral-7B-Instruct-v0.2"

model = hf_llm_boiler(model_id, peft=True)

Load function recognized for dfurman/Mistral-7B-Instruct-v0.2: mistral_loader
Run function recognized for dfurman/Mistral-7B-Instruct-v0.2: mistral


(…)ct-v0.2/resolve/main/adapter_config.json:   0%|          | 0.00/520 [00:00<?, ?B/s]

(…)-v0.2/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

(…)nstruct-v0.2/resolve/main/tokenizer.json:   0%|          | 0.00/1.79M [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)v0.1/resolve/main/generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

In [9]:
print(model.name, "\n")
print(model.tokenizer, "\n")
print(model.model, "\n")

mistral 

LlamaTokenizerFast(name_or_path='dfurman/Mistral-7B-Instruct-v0.2', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
} 

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear(
             

## Run the model

* For text generation options, refer to [https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.TextGenerationPipeline](https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.TextGenerationPipeline)
* Below prompts are borrowed from [https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md](https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md)

### Example 1: Zero-shot reasoning conditioned on good performance
* From https://arxiv.org/abs/2205.11916

In [10]:
# run zero shot questions

for itr, prompt in enumerate(pop_quiz["prompts"]["zero_shot"]):
    print(f"Question 1.{itr+1}")
    start_time = time.time()
    generated_text = model.run(
        prompt=prompt,
        eos_token_ids=model.tokenizer.eos_token_id,
        max_new_tokens=1024,
        temperature=0.01,
        do_sample=True,
        top_p=0.92,
        top_k=50,
        repetition_penalty=1.1,
        num_return_sequences=1,
        debug=True,
    )
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Question 1.1
*** Prompt
<s>[INST] A juggler has 16 balls. Half of the balls are golf balls and half of the golf balls are blue. How many blue golf balls are there?
Let's respond succinctly and work this out in a step by step way to be sure we have the right answer. [/INST] 
*** Generated
4 blue golf balls
--- 2.231147289276123 seconds ---


Question 1.2
*** Prompt
<s>[INST] Daniel is in need of a haircut. His barber works Mondays, Wednesdays, and Fridays. So, Daniel went in for a haircut on Sunday. Does this make logical sense?
Let's respond succinctly and work this out in a step by step way to be sure we have the right answer. [/INST] 
*** Generated
1. The problem states that Daniel's barber works only on Mondays, Wednesdays, and Fridays.
2. However, Daniel got his haircut on Sunday.
3. Since the barber doesn't work on Sundays, it does not logically make sense for Daniel to get his haircut on Sunday.
--- 4.211205720901489 seconds ---


Question 1.3
*** Prompt
<s>[INST] The cafeteria h

### Example 2: Least to most prompting
* From https://arxiv.org/abs/2205.10625


In [11]:
# run least to most questions

for itr, prompts in enumerate(pop_quiz["prompts"]["least_to_most"]):
    sub_question_1 = prompts[0]
    print(f"Question 2.{itr+1}")
    # Start with sub question #1

    start_time = time.time()
    res_1 = model.run(
        prompt=sub_question_1,
        eos_token_ids=model.tokenizer.eos_token_id,
        max_new_tokens=1024,
        temperature=0.01,
        do_sample=True,
        top_p=0.92,
        top_k=50,
        repetition_penalty=1.1,
        num_return_sequences=1,
        debug=True,
    )
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

    # Now do sub question #2 by appending answer to sub question #1
    sub_question_2 = f"{sub_question_1}\n{res_1}\n{prompts[1]}"

    start_time = time.time()
    res_2 = model.run(
        prompt=sub_question_2,
        eos_token_ids=model.tokenizer.eos_token_id,
        max_new_tokens=1000,
        temperature=0.01,
        do_sample=True,
        top_p=0.92,
        top_k=50,
        repetition_penalty=1.1,
        num_return_sequences=1,
        debug=True,
    )
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

Question 2.1
*** Prompt
<s>[INST] It takes Amy 4 minutes to climb to the top of a slide. It takes her 1 minute to slide down. How long does each trip take?
Let's respond succinctly and work this out in a step by step way to be sure we have the right answer. [/INST] 
*** Generated
1. First, let's calculate how long it takes for Amy to complete one full trip on the slide. This includes climbing up and sliding down.
2. Climbing up takes 4 minutes, and sliding down takes 1 minute. So, the total time for one trip is 4 + 1 = 5 minutes.
3. Therefore, each trip takes 5 minutes.
--- 4.9440484046936035 seconds ---


*** Prompt
<s>[INST] It takes Amy 4 minutes to climb to the top of a slide. It takes her 1 minute to slide down. How long does each trip take?
Let's respond succinctly and work this out in a step by step way to be sure we have the right answer.
1. First, let's calculate how long it takes for Amy to complete one full trip on the slide. This includes climbing up and sliding down.
2. Cl

### Example 3: Tab-CoT

* See https://arxiv.org/abs/2305.17812

In [12]:
# run tab-cot questions

for itr, prompts in enumerate(pop_quiz["prompts"]["tab_cot"]):
    print(f"Question 3.{itr+1}")
    # Start with sub question #1
    sub_question_1 = prompts[0]

    start_time = time.time()
    res_1 = model.run(
        prompt=sub_question_1,
        eos_token_ids=model.tokenizer.eos_token_id,
        max_new_tokens=1024,
        temperature=0.01,
        do_sample=True,
        top_p=0.92,
        top_k=50,
        repetition_penalty=1.1,
        num_return_sequences=1,
        debug=True,
    )
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

Question 3.1
*** Prompt
<s>[INST] Jackson is planting tulips. He can fit 6 red tulips in a row and 8 blue tulips in a row. If Jackson buys 36 red tulips and 24 blue tulips, how many rows of flowers will he plant?
Respond as succinctly as possible. Format the response as a completion of this table.
|step|subquestion|procedure|result|
|:---|:----------|:--------|:-----:| [/INST] 
*** Generated
1. How many red tulips can Jackson plant in one row? | 6 |  |
2. How many blue tulips can Jackson plant in one row? | 8 |  |
3. How many red tulips does Jackson have to plant? | 36 | Multiply 36 by 1 (since there's only one way to arrange them) |
4. How many blue tulips does Jackson have to plant? | 24 | Multiply 24 by 1 (since there's only one way to arrange them) |
5. What is the greatest common factor (GCF) of the number of red tulips and blue tulips? | 8 | Use Euclidean algorithm or prime factorization method |
6. How many full rows of red tulips can Jackson plant with his current supply? | 4.5